# Trainieren eines Klassifizierungsmodells mit automatisiertem maschinellen Lernen

Es gibt viele Arten von Machine Learning-Algorithmen, mit denen Sie ein Modell trainieren können, und manchmal ist es nicht einfach, den effektivsten Algorithmus für Ihre speziellen Daten und Vorhersageanforderungen zu bestimmen. Darüber hinaus können Sie die Vorhersageleistung eines Modells erheblich beeinflussen, indem Sie die Trainingsdaten mithilfe von Techniken wie beispielsweise der Normalisierung und Imputation fehlender Features vorverarbeiten. Um das beste Modell für Ihre Anforderungen zu finden, müssen Sie möglicherweise viele Kombinationen von Algorithmen und Vorverarbeitungstransformationen ausprobieren, was viel Zeit und Computeressourcen erfordert.

Mit Azure Machine Learning können Sie den Vergleich von Modellen automatisieren, die mit verschiedenen Algorithmen und Vorverarbeitungsoptionen trainiert wurden. Sie können die grafische Benutzeroberfläche in [Azure Machine Learning Studio](https://ml/azure.com) oder das Python SDK (v2) verwenden, um diese Funktionalität zu nutzen. Das Python SDK bietet Ihnen eine bessere Kontrolle über die Einstellungen für den Auftrag zum automatisierten maschinellen Lernen, aber die grafische Benutzeroberfläche ist benutzerfreundlicher.

## Vorbereitung

Sie benötigen die neueste Version des Pakets **azureml-ai-ml**, um den Code in diesem Notebook auszuführen. Führen Sie die folgende Zelle aus, um zu überprüfen, ob das Paket installiert ist.

> **Hinweis**:
> Wenn das Paket **azure-ai-ml** nicht installiert ist, führen Sie `pip install azure-ai-ml` aus, um es zu installieren.

In [ ]:
## Herstellen einer Verbindung mit Ihrem Arbeitsbereich

Sie können nun eine Verbindung mit Ihrem Arbeitsbereich herstellen, nachdem Sie die erforderlichen SDK-Pakete installiert haben.

Um eine Verbindung mit einem Arbeitsbereich herzustellen, benötigen Sie Bezeichnerparameter: eine Abonnement-ID, einen Ressourcengruppennamen und einen Arbeitsbereichsnamen. Der Ressourcengruppenname und Arbeitsbereichsname sind bereits für Sie ausgefüllt. Sie müssen lediglich die Abonnement-ID angeben, um den Befehl zu vervollständigen.

Um die erforderlichen Parameter zu finden, klicken Sie rechts oben in Studio auf das Abonnement und den Namen des Arbeitsbereichs. Rechts wird ein Bereich geöffnet.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Kopieren Sie die Abonnement-ID, und ersetzen Sie **YOUR-SUBSCRIPTION-ID** durch den Wert, den Sie kopiert haben. </p>

## Vorbereiten von Daten

Sie müssen kein Trainingsskript für automatisiertes maschinelles Lernen erstellen, aber Sie müssen die Trainingsdaten laden. 

In diesem Fall verwenden Sie ein Dataset mit Details zu Diabetespatienten. 

Um ein Dataset als Eingabe an einen Auftrag zum automatisierten maschinellen Lernen zu übergeben, müssen die Daten in tabellarischer Form vorliegen und eine Zielspalte enthalten. Damit die Daten als tabellarisches Dataset interpretiert werden können, muss das Eingabedataset eine **MLTable** sein.

Während der Einrichtung wurde bereits eine MLTable-Datenressource für Sie erstellt. Sie können die Datenressource erkunden, indem Sie zur Seite **Daten** navigieren. Sie rufen die Datenressource hier ab, indem Sie ihren Namen (`diabetes-training-table`) und ihre Version (`1`) angeben. 

In [ ]:
## Konfigurieren des Auftrags zum automatisierten maschinellen Lernen

Nun können Sie das Experiment für automatisiertes maschinelles Lernen konfigurieren.

Wenn Sie den folgenden Code ausführen, wird ein Auftrags zum automatisierten maschinellen Lernen erstellt:

- Verwendet den Computecluster namens `aml-cluster`
- Legt `Diabetic` als Zielspalte fest
- Legt `accuracy` als primäre Metrik fest
- Führt zu einem Timeout nach `60` Minuten der gesamten Trainingszeit 
- Trainiert maximal `5` Modelle
- Kein Modell wird mit dem Algorithmus `LogisticRegression` trainiert

In [ ]:
## Ausführen eines Auftrags für automatisiertes maschinelles Lernen

Es kann losgehen! Führen Sie das Experiment für automatisiertes maschinelles Lernen durch.

> **Hinweis**: Dieser Vorgang kann einige Zeit dauern!

Während der Ausführung des Auftrags können Sie ihn in Studio überwachen.

In [ ]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input

# creates a dataset based on the files in the local data folder
my_training_data_input = Input(type=AssetTypes.MLTABLE, path="azureml:diabetes-training:1")

## Configure automated machine learning job

Now, you're ready to configure the automated machine learning experiment.

When you run the code below, it will create an automated machine learning job that:

- Uses the compute cluster named `aml-cluster`
- Sets `Diabetic` as the target column
- Sets `accuracy` as the primary metric
- Times out after `60` minutes of total training time 
- Trains a maximum of `5` models
- No model will be trained with the `LogisticRegression` algorithm

In [ ]:
from azure.ai.ml import automl

# configure the classification job
classification_job = automl.classification(
    compute="aml-cluster",
    experiment_name="auto-ml-class-dev",
    training_data=my_training_data_input,
    target_column_name="Diabetic",
    primary_metric="accuracy",
    n_cross_validations=5,
    enable_model_explainability=True
)

# set the limits (optional)
classification_job.set_limits(
    timeout_minutes=60, 
    trial_timeout_minutes=20, 
    max_trials=5,
    enable_early_termination=True,
)

# set the training properties (optional)
classification_job.set_training(
    blocked_training_algorithms=["LogisticRegression"], 
    enable_onnx_compatible_models=True
)

## Run an automated machine learning job

OK, you're ready to go. Let's run the automated machine learning experiment.

> **Note**: This may take some time!

In [ ]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  

# submit the job to the backend
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

While the job is running, you can monitor it in the Studio.